In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as ticker
from matplotlib import rc
import scipy.integrate as integrate
from tqdm import tqdm

In [4]:
# Matplotlib customize
plt.rcParams['figure.figsize'] = [6,5]
plt.rcParams['figure.dpi'] = 100

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20

plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['lines.markersize'] = 4

plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.markerscale'] = 1.5
plt.rcParams['legend.borderpad'] = 0.6
plt.rcParams['legend.framealpha'] = 0.7

In [5]:
class BayesianCalculator:
    
    # Channels
    def __init__(self,channels):
        
        self.n = channels[:,0]
        self.b = channels[:,1]
        self.s = channels[:,2]
        
        # NChannels
        self.N = channels.shape[0]
        
    def PrintChannels(self):
        for i in range(self.N):
            print(i,':',self.n[i],self.b[i],self.s[i])
    
    
    

In [6]:
# Prueba
channels = np.array([[2.,3.5,1.]])

In [7]:
b1 = BayesianCalculator(channels)

In [8]:
b1.PrintChannels()

0 : 2.0 3.5 1.0
